In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
colabdir = '/content/drive/My Drive/Colab Notebooks'
import os
import pandas as pd
df = pd.read_csv(os.path.join(colabdir,'fra.txt'), names=['eng','fra'], sep='\t', header=None)
len(df)

170190

In [0]:
df.columns = ['eng', 'fra']

In [4]:
df.tail()

,eng,fra
170185,A carbon footprint is the amount of carbon dio...,Une empreinte carbone est la somme de pollutio...
170186,Death is something that we're often discourage...,La mort est une chose qu'on nous décourage sou...
170187,Since there are usually multiple websites on a...,Puisqu'il y a de multiples sites web sur chaqu...
170188,If someone who doesn't know your background sa...,Si quelqu'un qui ne connaît pas vos antécédent...
170189,It may be impossible to get a completely error...,Il est peut-être impossible d'obtenir un Corpu...


In [5]:
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [0]:
eng_tok = Tokenizer(char_level=True)

In [0]:
eng_tok.fit_on_texts(df['eng'].tolist())

In [0]:
eng_data = eng_tok.texts_to_sequences(df['eng'].tolist())

In [0]:
fra_tok = Tokenizer(char_level=True)
fra_tok.fit_on_texts(df['fra'].tolist())
fra_data = fra_tok.texts_to_sequences(df['fra'].tolist())

In [0]:
fra_tok.word_index['<START>'] = len(fra_tok.word_index) + 1
fra_tok.word_index['<END>'] = len(fra_tok.word_index) + 1

In [0]:
new_fra_data = []
for s in fra_data:
    new_s = [fra_tok.word_index['<START>']] + s + [fra_tok.word_index['<END>']]
    new_fra_data.append(new_s)
fra_data = new_fra_data

In [0]:
NUM_ENG_CHAR = len(eng_tok.word_index) + 1
NUM_FRA_CHAR = len(fra_tok.word_index) + 1

In [0]:
from keras.preprocessing.sequence import pad_sequences

In [0]:
eng_pad = pad_sequences(eng_data, padding='pre')
fra_pad = pad_sequences(fra_data, padding='post')

In [19]:
from keras.models import Model
from keras.layers import Dense, Embedding, Input, LSTM

enc_input = Input(shape=(None, NUM_ENG_CHAR))
encoder = LSTM(16, return_state=True, return_sequences=False)
enc_output, state_h, state_c = encoder(enc_input)
enc_state = [state_h, state_c]

In [0]:
dec_input = Input(shape=(None, NUM_FRA_CHAR))
decoder = LSTM(16, return_sequences=True, return_state=True)
dec_output, _, _ = decoder(dec_input, initial_state=enc_state)
predict_layer = Dense(NUM_FRA_CHAR, activation='softmax')
next_char = predict_layer(dec_output)
model = Model([enc_input, dec_input], next_char)

In [0]:
from keras.utils import Sequence
import numpy as np

class OneHotSequence(Sequence):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.batch_size = 32

    def __len__(self):
        return len(self.x) // self.batch_size

    def __getitem__(self, idx):
        start = idx * self.batch_size
        end = start + self.batch_size

        # one-hot encoding
        x = np.eye(NUM_ENG_CHAR)[self.x[start:end,]]
        y = np.eye(NUM_FRA_CHAR)[self.y[start:end,]]

        target = np.expand_dims(self.y[start:end,], 2)

        return [x, y], target

In [22]:
seq = OneHotSequence(eng_pad, fra_pad)
len(seq)

5318

In [23]:
colabdir = '/content/drive/My Drive/Colab Notebooks'
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
model.fit_generator(seq, epochs=1)
model.save(os.path.join(colabdir,'seq2seq.krs'))



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/1
5318/5318 [==============================] - 5301s 997ms/step - loss: 0.1720


In [0]:
encoder_model = Model(enc_input, enc_state)

In [0]:
prev_h = Input(shape=(16,))
prev_c = Input(shape=(16,))

prev_state = [prev_h, prev_c]
 
dec_output, current_h, current_c = decoder(dec_input, initial_state=prev_state)
current_state = [current_h, current_c]

pred_char = predict_layer(dec_output)
decoder_model = Model([dec_input] + prev_state, [pred_char] + current_state)

In [0]:
_, ENG_LENGTH = eng_pad.shape
x = eng_tok.texts_to_sequences(["This class is great!"])
x = pad_sequences(x, ENG_LENGTH, padding='pre')
x = np.eye(NUM_ENG_CHAR)[x]

In [54]:
NUM_FRA_CHAR

84

In [0]:
state = encoder_model.predict(x)
target_sentence = ''
target_seq = np.eye(NUM_FRA_CHAR)[fra_tok.word_index['<START>']]
target_seq = np.array([[target_seq]])
output_token, h, c = decoder_model.predict([target_seq] + state)
token_index = output_token[0, -1, :].argmax()
char = fra_tok.index_word[token_index-1]
target_sentence += char

In [0]:
while token_index != fra_tok.word_index['<END>']:
    target_seq = np.eye(NUM_FRA_CHAR)[token_index]
    target_seq = np.array([[target_seq]])

    output_token, h, c = decoder_model.predict([target_seq, h, c])

    token_index = output_token[0, -1, :].argmax()

    char = fra_tok.index_word[token_index-1]
    target_sentence += char

    # 무한 루프 방지를 위해 300글자가 넘으면 강제 중단한다.
    if len(target_sentence) > 300:
        break

In [0]:
# 왤케안되누